In [ ]:
#|export
import subprocess
import dspy
import logging

In [ ]:
import ollama


In [ ]:
# |export
########## DSPy Config  ##########
llm = dspy.OllamaLocal("nous-llama-3-8b-instruct", max_tokens=4000, model_type="text")
dspy.settings.configure(lm=llm)

In [ ]:
llm("Hi, how are you?")

In [ ]:
########## Arize Phoenix ##########
import phoenix as px
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

In [ ]:
phoenix_session = px.launch_app()
endpoint = "http://localhost:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

In [ ]:
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()
class TestTrace(dspy.Signature):
    """
    Perform the task requested to the best of your ability.
    """

    task = dspy.InputField(desc="Task to be performed.")
    answer = dspy.OutputField(desc="The answer.")

In [ ]:
test_trace = dspy.Predict(TestTrace)
answer = test_trace(task="Say Hello.")
test_trace_cot = dspy.ChainOfThought(TestTrace)
test_trace_cot(task="Write a python function that adds two numbers together.")

In [ ]:
cladio_binary = test_trace_cot(task="Write a short story about a cat named Claudio that quickly devolves into just python code and then just binary.")

In [ ]:
cladio_binary.answer